# JapDic 
Version 2.0
Made by Vik

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from notion_client import Client
import os
from dotenv import load_dotenv

In [2]:
load_dotenv("locals.env")
token = os.getenv("NOTION_TOKEN")
table_id = os.getenv("NOTION_TABLE_ID")
page_id = os.getenv("NOTION_JAPDIC_DEMO_PAGE_ID")

In [3]:
# database_id = os.getenv("NOTION_DATABASE_ID")

client = Client(auth=token)
page = client.pages.retrieve(page_id=page_id)

## Add header case

In [4]:
# Function to add text to a page
def add_text_to_page(page_id: str, text: str):
    """
    Adds text as a paragraph block to a Notion page

    Args:
        page_id (str): The ID of the Notion page
        text (str): The text content to add
    """
    try:
        response = client.blocks.children.append(
            block_id=page_id,
            children=[
                {
                    "type": "paragraph",
                    "paragraph": {
                        "rich_text": [
                            {"type": "text", "text": {"content": text, "link": None}}
                        ],
                        "color": "default",
                    },
                }
            ],
        )
        return response
    except Exception as e:
        print(f"Error adding text to page: {e}")
        return None

## Table

In [5]:
def check_for_existing_table(page_id: str) -> str | None:
    """
    Checks if a table already exists on the page

    Args:
        page_id (str): The ID of the Notion page

    Returns:
        str | None: Table ID if found, None otherwise
    """
    try:
        # Get all blocks on the page
        response = client.blocks.children.list(block_id=page_id)

        # Look for a table block
        for block in response["results"]:
            if block["type"] == "table":
                return block["id"]

        return None
    except Exception as e:
        print(f"Error checking for existing table: {e}")
        return None

In [6]:
def create_japanese_table(page_id: str):
    """
    Creates a table with columns: ID, ROOMA-JI, Pronounced, English

    Args:
        page_id (str): The ID of the parent page where the table will be created
    """
    try:
        response = client.blocks.children.append(
            block_id=page_id,
            children=[
                {
                    "type": "table",
                    "table": {
                        "table_width": 4,
                        "has_column_header": True,
                        "has_row_header": False,
                        "children": [
                            # Header row
                            {
                                "type": "table_row",
                                "table_row": {
                                    "cells": [
                                        [{"type": "text", "text": {"content": "ID"}}],
                                        [
                                            {
                                                "type": "text",
                                                "text": {"content": "ROOMA-JI"},
                                            }
                                        ],
                                        [
                                            {
                                                "type": "text",
                                                "text": {"content": "Pronounced"},
                                            }
                                        ],
                                        [
                                            {
                                                "type": "text",
                                                "text": {"content": "English"},
                                            }
                                        ],
                                    ]
                                },
                            }
                            # You can add more rows here if needed
                        ],
                    },
                }
            ],
        )
        return response
    except Exception as e:
        print(f"Error creating table: {e}")
        return None


In [7]:
def get_or_create_table(page_id: str) -> str | None:
    """
    Gets existing table ID or creates new table

    Args:
        page_id (str): The ID of the Notion page

    Returns:
        str | None: Table ID if found/created, None if failed
    """
    # First check for existing table
    table_id = check_for_existing_table(page_id)

    if table_id:
        print("Found existing table")
        return table_id

    # If no table exists, create new one
    print("Creating new table")
    response = create_japanese_table(page_id)
    if response and "results" in response:
        table_id = response["results"][0]["id"]

        # Store the table ID in .env file
        with open("locals.env", "a") as f:
            f.write(f"\nNOTION_TABLE_ID={table_id}")

        return table_id

    return None

In [8]:
def add_table_row(table_id: str, id_num: int, rooma_ji: str, english: str):
    """
    Adds a new row to the table

    Args:
        table_id (str): The ID of the table
        id_num (int): The ID number
        rooma_ji (str): The romanized Japanese text
        english (str): The English translation
    """
    try:
        response = client.blocks.children.append(
            block_id=table_id,
            children=[
                {
                    "type": "table_row",
                    "table_row": {
                        "cells": [
                            [{"type": "text", "text": {"content": str(id_num)}}],
                            [{"type": "text", "text": {"content": rooma_ji}}],
                            [
                                {"type": "text", "text": {"content": ""}}
                            ],  # Empty Pronounced cell
                            [{"type": "text", "text": {"content": english}}],
                        ]
                    },
                }
            ],
        )
        return response
    except Exception as e:
        print(f"Error adding row: {e}")
        return None

In [9]:
def get_last_row_id(table_id: str) -> int:
    """
    Gets the ID from the last row in the table

    Args:
        table_id (str): The ID of the table

    Returns:
        int: The last ID used in the table, or 0 if table is empty
    """
    try:
        # Get all rows from the table
        response = client.blocks.children.list(block_id=table_id)

        # Skip the header row and get the last row
        rows = response["results"]
        if len(rows) <= 1:  # Only header row or empty table
            return 0

        last_row = rows[-1]
        if last_row["type"] == "table_row":
            # Get the ID from the first cell
            id_cell = last_row["table_row"]["cells"][0]
            if id_cell and len(id_cell) > 0:
                return int(id_cell[0]["text"]["content"])

        return 0

    except Exception as e:
        print(f"Error getting last row ID: {e}")
        return 0

In [10]:
# Get or create table if needed
if not table_id:
    table_id = get_or_create_table(page_id)
    print(table_id)
    if not table_id:
        print("Failed to get or create table")
        exit(1)

Creating new table
143ebe12-09f2-815d-bb44-d46a7c0a0b48


Code to add rows to table:

In [17]:
# Read the words from the text file
with open("data/orig_demo.txt", "r") as f:
    next_id = get_last_row_id(table_id) + 1
    for line in f:
        words = line.strip().split(" = ")
        english_word = words[0]
        romaji_word = words[1]
        page_title = f"{english_word} - {romaji_word}"
        # Now you can use the table_id for adding rows
        add_table_row(table_id, next_id, romaji_word, english_word)
        next_id += 1

## Tests

In [12]:
# Test function to verify word parsing
def test_parse_word_line():
    # Test case 1: Normal case
    test_line = "hello,konnichiwa"
    english, romaji = test_line.strip().split(",")
    assert english == "hello"
    assert romaji == "konnichiwa"

    # Test case 2: With spaces
    test_line = "good morning, ohayou gozaimas"
    english, romaji = test_line.strip().split(",")
    assert english == "good morning"
    assert romaji == " ohayou gozaimas"

    # Test case 3: Empty values
    test_line = ","
    english, romaji = test_line.strip().split(",")
    assert english == ""
    assert romaji == ""


# Run the test
test_parse_word_line()
print("All tests passed!")


All tests passed!
